In [ ]:
##!pip install ipywidgets
from google.colab import output
output.enable_custom_widget_manager()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from scipy.optimize import least_squares
from scipy.signal import fftconvolve
import ipywidgets as widgets
from ipywidgets import interact, FloatSlider, IntSlider

# ------------------------------
# PARAMÈTRES DE BASE
# ------------------------------
np.random.seed(42)
fs = 4000.0               # fréquence d'échantillonnage
duration = 1.0
N = int(fs * duration)
t = np.arange(N) / fs
c_default = 1400.0        # vitesse du son dans l'eau

# ------------------------------
# 1) Générer un réseau d’eau simulé
# ------------------------------
G = nx.random_geometric_graph(40, radius=0.26)
pos = nx.get_node_attributes(G, 'pos')
coords = np.array([pos[i] for i in range(len(pos))])

sensor_indices = [2, 8, 15, 27]
sensor_pos = coords[sensor_indices]

# ------------------------------
# SIGNAL DE FUITE (impulsion)
# ------------------------------
def make_pulse(t, freq=35, width=0.08):
    envelope = np.exp(-((t - 0.2)/width)**2)
    carrier = np.sin(2*np.pi*freq*t)
    return envelope * carrier

base_signal = make_pulse(t)


# ------------------------------
# TDOA avec GCC-PHAT
# ------------------------------
def gcc_phat(x, y, fs, max_tau=0.05):
    n = x.shape[0] + y.shape[0]
    X = np.fft.rfft(x, n=n)
    Y = np.fft.rfft(y, n=n)
    R = X * np.conj(Y)

    denom = np.abs(R)
    denom[denom == 0] = 1e-12
    R /= denom

    cc = np.fft.irfft(R, n=n)
    max_shift = min(int(fs * max_tau), len(cc)//2)
    cc = np.concatenate((cc[-max_shift:], cc[:max_shift+1]))

    shift = np.argmax(np.abs(cc)) - max_shift
    tau = shift / float(fs)
    return tau


# ------------------------------
# Multilatération
# ------------------------------
def multilaterate(delta_d, sensors, ref, x0):
    def residuals(vars):
        x, y = vars
        dists = np.linalg.norm(sensors - np.array([x,y]), axis=1)
        return (dists - dists[ref]) - delta_d
    sol = least_squares(residuals, x0)
    return sol.x


# ------------------------------
# FONCTION DE MISE À JOUR INTERACTIVE
# ------------------------------
def update(x_real=0.6, y_real=0.3, noise=-6.0, speed=1400.0):

    real_pos = np.array([x_real, y_real])
    num_sensors = len(sensor_pos)

    # 1) Générer signaux
    signals = []
    for sp in sensor_pos:
        dist = np.linalg.norm(real_pos - sp)
        delay = dist / speed

        delay_samples = int(np.round(delay * fs))
        sig = np.zeros_like(base_signal)
        if delay_samples < len(sig):
            sig[delay_samples:] = base_signal[:len(sig)-delay_samples]

        Ps = np.mean(sig**2)
        noise_power = Ps / (10**(noise / 10 + 1e-12))
        sig += np.random.normal(0, np.sqrt(noise_power), sig.shape)
        signals.append(sig)
    signals = np.array(signals)

    # 2) TDOA via GCC-PHAT
    ref = 0
    tdoas = []
    for i in range(num_sensors):
        if i == ref:
            tdoas.append(0.0)
        else:
            tdoas.append(gcc_phat(signals[ref], signals[i], fs))
    tdoas = np.array(tdoas)
    delta_d = speed * tdoas

    # 3) Estimation via multilateration
    est_pos = multilaterate(delta_d, sensor_pos, ref, x0=sensor_pos.mean(axis=0))
    error = np.linalg.norm(est_pos - real_pos)

    # 4) Affichage graphique
    plt.figure(figsize=(8,8))
    nx.draw_networkx_edges(G, pos, alpha=0.20)
    nx.draw_networkx_nodes(G, pos, node_color='lightgray', node_size=35)

    plt.scatter(sensor_pos[:,0], sensor_pos[:,1], marker='^', s=200, label="Capteurs")
    plt.scatter(real_pos[0], real_pos[1], s=260, color='green', marker='*', label="Fuite réelle")
    plt.scatter(est_pos[0], est_pos[1], s=220, color='red', marker='X', label="Fuite estimée")

    plt.title(f"Détection de fuite — Erreur = {error:.3f} m", fontsize=14)
    plt.legend()
    plt.gca().set_aspect('equal')
    plt.show()

    print("Position réelle  :", real_pos)
    print("Position estimée :", est_pos)
    print("Erreur (m)       :", error)


# ------------------------------
# SLIDERS INTERACTIFS
# ------------------------------
interact(
    update,
    x_real=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.6, description="Fuite X"),
    y_real=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.3, description="Fuite Y"),
    noise=FloatSlider(min=-20, max=20, step=1, value=-6, description="SNR (dB)"),
    speed=FloatSlider(min=1000, max=2000, step=10, value=1400, description="Vitesse c"),
)


interactive(children=(FloatSlider(value=0.6, description='Fuite X', max=1.0, step=0.01), FloatSlider(value=0.3…

<function __main__.update(x_real=0.6, y_real=0.3, noise=-6.0, speed=1400.0)>